## Load model

In [39]:
import torch
from eval import Classifier

model = Classifier()
model.load_state_dict(torch.load("./models/model.pth"))

<All keys matched successfully>

In [40]:
model

Classifier(
  (cnn): Sequential(
    (0): Conv1d(8, 10, kernel_size=(3,), stride=(1,))
    (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Flatten(start_dim=1, end_dim=-1)
  )
  (fc): Sequential(
    (0): Linear(in_features=30, out_features=27, bias=True)
  )
)

In [41]:
import numpy as np

def tensor_to_hex_list(tensor: torch.Tensor, float_bits: int, bits: int):
    arr = tensor.detach().cpu().numpy()
    arr = arr * 2 ** float_bits
    arr = np.array(arr.reshape(-1), dtype=np.int64)
    hex_list =  [f"{bits}'h" + hex(abs(i))[2:].rjust(int(bits/4), "0") if i > 0 else f"-{bits}'h" + hex(abs(i))[2:].rjust(int(bits/4), "0") for i in arr]
    
    return hex_list

## Get params

### Get conv params

In [42]:
model.cnn[0].weight

Parameter containing:
tensor([[[-19.6986, -21.7953, -27.0025],
         [ 44.4133,  44.1890,  44.3681],
         [-11.4387,  -3.3286,   3.5227],
         [-41.0221, -40.3241, -40.9374],
         [ 32.3187,  32.3442,  31.1909],
         [-41.2163, -41.4004, -41.5270],
         [ -0.6060,  -0.5368,  -0.0994],
         [ 25.2135,  24.7958,  24.0633]],

        [[-18.5927, -15.7875, -10.1012],
         [-22.4407,  -9.6915, -16.2810],
         [-19.5670, -17.0767, -19.1395],
         [ 16.2055,  17.0452,  15.2373],
         [-37.2768, -37.7453, -38.3550],
         [-44.4583, -44.1875, -44.3386],
         [ -5.2725,  -6.0417,  -7.7615],
         [  7.1311,   9.4710,  10.7750]],

        [[ 29.1816,  25.8611,  25.5599],
         [ 25.9636,  21.6094,  31.7821],
         [-14.1536,  -0.4081, -11.8292],
         [ -2.6685,   5.0790,   1.2523],
         [-18.4982, -20.3428, -22.7023],
         [-40.6813, -40.6730, -40.3232],
         [ 43.1092,  42.9827,  43.3872],
         [ 43.9095,  43.9275,  

In [43]:
model.cnn[0].bias

Parameter containing:
tensor([  8.8471,  11.4942, -11.0057,  20.3848,  35.9173, -30.4756,  39.6558,
         -3.6994, -40.6424,  39.3213], requires_grad=True)

In [44]:
kernel_weights = []

for i in model.cnn[0].weight:
    kernel_weights.append(tensor_to_hex_list(i, float_bits=8, bits=16))

In [45]:
kernel_weights

[["-16'h13b2",
  "-16'h15cb",
  "-16'h1b00",
  "16'h2c69",
  "16'h2c30",
  "16'h2c5e",
  "-16'h0b70",
  "-16'h0354",
  "16'h0385",
  "-16'h2905",
  "-16'h2852",
  "-16'h28ef",
  "16'h2051",
  "16'h2058",
  "16'h1f30",
  "-16'h2937",
  "-16'h2966",
  "-16'h2986",
  "-16'h009b",
  "-16'h0089",
  "-16'h0019",
  "16'h1936",
  "16'h18cb",
  "16'h1810"],
 ["-16'h1297",
  "-16'h0fc9",
  "-16'h0a19",
  "-16'h1670",
  "-16'h09b1",
  "-16'h1047",
  "-16'h1391",
  "-16'h1113",
  "-16'h1323",
  "16'h1034",
  "16'h110b",
  "16'h0f3c",
  "-16'h2546",
  "-16'h25be",
  "-16'h265a",
  "-16'h2c75",
  "-16'h2c30",
  "-16'h2c56",
  "-16'h0545",
  "-16'h060a",
  "-16'h07c2",
  "16'h0721",
  "16'h0978",
  "16'h0ac6"],
 ["16'h1d2e",
  "16'h19dc",
  "16'h198f",
  "16'h19f6",
  "16'h159c",
  "16'h1fc8",
  "-16'h0e27",
  "-16'h0068",
  "-16'h0bd4",
  "-16'h02ab",
  "16'h0514",
  "16'h0140",
  "-16'h127f",
  "-16'h1457",
  "-16'h16b3",
  "-16'h28ae",
  "-16'h28ac",
  "-16'h2852",
  "16'h2b1b",
  "16'h2afb",
  "1

In [46]:
bias = tensor_to_hex_list(model.cnn[0].bias, 8, 16)
bias

["16'h08d8",
 "16'h0b7e",
 "-16'h0b01",
 "16'h1462",
 "16'h23ea",
 "-16'h1e79",
 "16'h27a7",
 "-16'h03b3",
 "-16'h28a4",
 "16'h2752"]

In [142]:
with open("cnn_param.txt", "w") as fp:
    # write kernel weights
    for i, weight in enumerate(kernel_weights):
        fp.write(f"localparam [15:0] kernel_weight{i} [0:23] = '{'{'}\n")
        for j, w in enumerate(weight[:-1]):
            if j % 8 == 0:
                fp.write(" " * 4)
            
            fp.write(f"{w}, ")
            
            if j % 8 == 7:
                fp.write("\n")
            
        fp.write(f" {weight[-1]}\n" + "};\n\n")
    
    # write bias
    fp.write("localparam [15:0] cnn_bias [0:9] = '{\n")
    fp.write(" " * 4)
    for i, b in enumerate(bias[:-1]):
        fp.write(f"{b}, ")
    fp.write(f"{bias[-1]}\n" + "};\n")

### Get fc param

In [48]:
model.fc[0].weight, model.fc[0].weight.shape

(Parameter containing:
 tensor([[  9.9009,   6.7030,  12.8561, -17.1859, -15.4959, -14.5686,  10.5488,
            9.6615,  11.1175,  13.5918,  12.2274,  12.0412, -13.4970, -10.8626,
           -7.1948,   8.3417,   8.2107,  12.0162,  -9.4247,  -9.1299,  -9.3204,
           -7.0088,  -6.6695,  -4.7617,  20.4395,  20.8245,  19.5263, -21.5311,
          -22.3276, -24.5059],
         [ -9.8423,  -8.2556,  -8.6066,  18.7477,  18.0189,  18.0065, -13.7677,
          -10.0600, -12.6918, -28.6141, -27.7226, -27.3027,   5.0098,   4.6335,
            1.5405,   4.2506,   6.7063,   5.2010,   8.0726,   5.3810,   2.6762,
           18.4038,  16.9606,  17.1018,  -2.3210,  -3.7809,  -4.4687,   2.4210,
            2.4838,   3.7457],
         [-11.5728,  -8.5998, -15.1555, -15.2280, -13.8639, -15.0411, -10.5546,
           -9.7625, -12.5296,   7.7000,   7.4855,   9.8387, -10.7691,  -9.3761,
          -10.3291, -11.7320,  -6.8195, -12.5887, -11.6014, -10.7547, -13.4409,
           -1.7092,   0.1532,  -3.4

In [49]:
model.fc[0].bias

Parameter containing:
tensor([-42.5195,   5.7998,  27.6298, -23.9818,  24.4148,  29.1349,  -8.1405,
         10.6412,   6.3522, -41.3713, -31.6411,  27.3438, -43.1333, -39.3329,
         45.0211, -18.4086, -38.9969, -11.5696,  -7.3386,  18.2612,  16.9376,
         21.9714, -22.5964, -36.8528, -23.7138,  34.5924, -44.0687],
       requires_grad=True)

In [50]:
fc_weights = []

for i in model.fc[0].weight:
    fc_weights.append(tensor_to_hex_list(i, 8, 16))

In [51]:
fc_weights

[["16'h09e6",
  "16'h06b3",
  "16'h0cdb",
  "-16'h112f",
  "-16'h0f7e",
  "-16'h0e91",
  "16'h0a8c",
  "16'h09a9",
  "16'h0b1e",
  "16'h0d97",
  "16'h0c3a",
  "16'h0c0a",
  "-16'h0d7f",
  "-16'h0adc",
  "-16'h0731",
  "16'h0857",
  "16'h0835",
  "16'h0c04",
  "-16'h096c",
  "-16'h0921",
  "-16'h0952",
  "-16'h0702",
  "-16'h06ab",
  "-16'h04c2",
  "16'h1470",
  "16'h14d3",
  "16'h1386",
  "-16'h1587",
  "-16'h1653",
  "-16'h1881"],
 ["-16'h09d7",
  "-16'h0841",
  "-16'h089b",
  "16'h12bf",
  "16'h1204",
  "16'h1201",
  "-16'h0dc4",
  "-16'h0a0f",
  "-16'h0cb1",
  "-16'h1c9d",
  "-16'h1bb8",
  "-16'h1b4d",
  "16'h0502",
  "16'h04a2",
  "16'h018a",
  "16'h0440",
  "16'h06b4",
  "16'h0533",
  "16'h0812",
  "16'h0561",
  "16'h02ad",
  "16'h1267",
  "16'h10f5",
  "16'h111a",
  "-16'h0252",
  "-16'h03c7",
  "-16'h0477",
  "16'h026b",
  "16'h027b",
  "16'h03be"],
 ["-16'h0b92",
  "-16'h0899",
  "-16'h0f27",
  "-16'h0f3a",
  "-16'h0ddd",
  "-16'h0f0a",
  "-16'h0a8d",
  "-16'h09c3",
  "-16'h0c8

In [52]:
fc_bias = tensor_to_hex_list(model.fc[0].bias, 8, 16)
fc_bias

["-16'h2a85",
 "16'h05cc",
 "16'h1ba1",
 "-16'h17fb",
 "16'h186a",
 "16'h1d22",
 "-16'h0823",
 "16'h0aa4",
 "16'h065a",
 "-16'h295f",
 "-16'h1fa4",
 "16'h1b58",
 "-16'h2b22",
 "-16'h2755",
 "16'h2d05",
 "-16'h1268",
 "-16'h26ff",
 "-16'h0b91",
 "-16'h0756",
 "16'h1242",
 "16'h10f0",
 "16'h15f8",
 "-16'h1698",
 "-16'h24da",
 "-16'h17b6",
 "16'h2297",
 "-16'h2c11"]

In [141]:
with open("fc_param.txt", "w") as fp:
    # write kernel weights
    for i, weight in enumerate(fc_weights):
        fp.write(f"localparam [15:0] fc_weight{i} [0:29] = '{'{'}\n")
        for j, w in enumerate(weight[:-1]):
            if j % 6 == 0:
                fp.write(" " * 4)
            
            fp.write(f"{w}, ")
            
            if j % 6 == 5:
                fp.write("\n")
            
        fp.write(f" {weight[-1]}\n" + "};\n\n")
    
    # write bias
    fp.write("localparam [15:0] fc_bias [0:26] = '{\n")
    for i, b in enumerate(fc_bias[:-1]):
        if i % 6 == 0:
            fp.write(" " * 4)
        fp.write(f"{b}, ")
        if i % 6 == 5:
            fp.write("\n")
    fp.write(f"{bias[-1]}\n" + "};\n")

In [54]:
mean = torch.tensor([
    773.01792745,
    -88.91943195,
    257.42027559,
    318.24409449,
    324.74732846,
    334.68848425,
    340.11283746,
    307.42055681,
])
std = torch.tensor([
    497.12221911,
    253.92590509,
    272.79684578,
    19.25624324,
    45.58375268,
    43.85550165,
    36.24868204,
    41.05464977,
])

In [55]:
mean_hex = tensor_to_hex_list(mean, 0, 16)
std_hex = tensor_to_hex_list(std, 0, 16)

In [56]:
for i in mean_hex:
    print(i + ",")

16'h0305,
-16'h0058,
16'h0101,
16'h013e,
16'h0144,
16'h014e,
16'h0154,
16'h0133,


In [57]:
for i in std_hex:
    print(i + ",")

16'h01f1,
16'h00fd,
16'h0110,
16'h0013,
16'h002d,
16'h002b,
16'h0024,
16'h0029,


In [90]:
from eval import NMSLTestDataset
from torch.utils.data import Dataset, DataLoader

train_set = NMSLTestDataset("../dataset/test/", "A.txt")
train_loader = DataLoader(train_set, batch_size=1, shuffle=False)


In [145]:
for batch in train_loader:
    i, l = batch
    break

In [93]:
model.fc[0](model.cnn[0](i).view(-1).unsqueeze(0))

tensor([[ 76868.6562, -65322.8789,  22524.3672,  37727.6836,  42806.6914,
         -43448.3945, -35938.4922,   8330.0566,    453.1064,   4230.6519,
         -39792.9922, -41313.3398, -47647.0742, -26779.8594,  28111.5605,
         -57592.0859, -49063.4570,  -4703.9258,  51697.7070,  70827.9609,
          -5346.2285, -11346.6787, -43768.6758,  56778.7109,  23025.4805,
          15439.6025, -56924.7578]], grad_fn=<AddmmBackward0>)

In [94]:
l

[tensor([-32])]

In [121]:
def normalize(data: torch.Tensor):
    mean = torch.tensor([
        773, -88, 257, 318, 324, 334, 340, 307,
    ])
    std = torch.tensor([
        497, 253, 272, 19, 45, 43, 36, 41,
    ])
    
    data_norm = torch.zeros((8, 5))
    
    for i in range(5):
        data_norm[:, i] = (data[:, i] - mean) / std
    
    return data_norm

In [122]:
data = torch.tensor([
    1012, 19, -3 , 303, 398, 403, 392, 347,
    1015, 23, -3 , 301, 397, 402, 395, 354,
    1012, 23, -11, 300, 394, 399, 391, 350,
    1012, 15, -7 , 301, 396, 401, 393, 351,
    1019, 31,  3 , 301, 396, 401, 393, 350,
])

In [126]:
norm = normalize(data.reshape((5, 8)).T)

In [131]:
model.fc[0](model.cnn[0](norm).view(-1))

tensor([ 78080.0156, -65960.9531,  23612.0859,  38308.7031,  43843.8828,
        -44652.2305, -36146.1055,   8917.1602,   1326.0968,   5286.1201,
        -39421.3516, -42218.4258, -47358.3281, -27976.0430,  29207.4082,
        -58933.7969, -50084.2930,  -6072.1631,  53210.4766,  71988.6328,
         -6518.6250, -12689.1650, -44850.7461,  57503.8750,  23961.7598,
         15686.6211, -57202.7656], grad_fn=<AddBackward0>)

In [125]:
model

Classifier(
  (cnn): Sequential(
    (0): Conv1d(8, 10, kernel_size=(3,), stride=(1,))
    (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Flatten(start_dim=1, end_dim=-1)
  )
  (fc): Sequential(
    (0): Linear(in_features=30, out_features=27, bias=True)
  )
)

In [132]:
model.cnn[0](norm)

tensor([[ 187.4835,  194.1343,  193.5637],
        [-408.6349, -406.0203, -405.7320],
        [ 123.5429,  131.0596,  127.5453],
        [ 320.2363,  327.2692,  319.2979],
        [  12.3275,   14.3208,   14.0996],
        [ 320.7651,  315.7285,  316.0187],
        [ -60.3004,  -66.5910,  -62.5255],
        [-134.2954, -140.8595, -138.8514],
        [ 333.8217,  330.2679,  331.7263],
        [ -59.3946,  -56.1558,  -57.5767]], grad_fn=<SqueezeBackward1>)

In [134]:
tensor_to_hex_list(model.cnn[0].weight[0], 8, 16)

["-16'h13b2",
 "-16'h15cb",
 "-16'h1b00",
 "16'h2c69",
 "16'h2c30",
 "16'h2c5e",
 "-16'h0b70",
 "-16'h0354",
 "16'h0385",
 "-16'h2905",
 "-16'h2852",
 "-16'h28ef",
 "16'h2051",
 "16'h2058",
 "16'h1f30",
 "-16'h2937",
 "-16'h2966",
 "-16'h2986",
 "-16'h009b",
 "-16'h0089",
 "-16'h0019",
 "16'h1936",
 "16'h18cb",
 "16'h1810"]

In [136]:
for i in tensor_to_hex_list(norm, 8, 16):
    print(f"{i},")

16'h007b,
16'h007c,
16'h007b,
16'h007b,
16'h007e,
16'h006c,
16'h0070,
16'h0070,
16'h0068,
16'h0078,
-16'h00f4,
-16'h00f4,
-16'h00fc,
-16'h00f8,
-16'h00ef,
-16'h00ca,
-16'h00e5,
-16'h00f2,
-16'h00e5,
-16'h00e5,
16'h01a4,
16'h019f,
16'h018e,
16'h0199,
16'h0199,
16'h019a,
16'h0194,
16'h0182,
16'h018e,
16'h018e,
16'h0171,
16'h0187,
16'h016a,
16'h0178,
16'h0178,
16'h00f9,
16'h0125,
16'h010c,
16'h0112,
16'h010c,


In [151]:
norm

tensor([[ 0.4809,  0.4869,  0.4809,  0.4809,  0.4950],
        [ 0.4229,  0.4387,  0.4387,  0.4071,  0.4704],
        [-0.9559, -0.9559, -0.9853, -0.9706, -0.9338],
        [-0.7895, -0.8947, -0.9474, -0.8947, -0.8947],
        [ 1.6444,  1.6222,  1.5556,  1.6000,  1.6000],
        [ 1.6047,  1.5814,  1.5116,  1.5581,  1.5581],
        [ 1.4444,  1.5278,  1.4167,  1.4722,  1.4722],
        [ 0.9756,  1.1463,  1.0488,  1.0732,  1.0488]])

In [152]:
model(norm.unsqueeze(0))

tensor([[  14.8787,  -27.4796,   53.5626,  -53.2382,   56.6895,   27.1135,
           15.5286,   -9.0373,   34.3900,   -4.5367,  -92.8308,   30.5288,
         -112.1741,  -21.3334,   84.3708, -109.3885,  -18.8583,  -22.2696,
           34.2130,   52.2408,   24.3375,   13.1511,  -72.2329,  -11.6911,
            9.4496,   28.0563,  -86.0589]], grad_fn=<AddmmBackward0>)